# Data

Nous disposons d'un ensemble de données de 17092 images de cellules normales du sang préphérique, les images sont répartis sur 8 classes de types distincts à savoir : 'basophil','eosinophil', 'erythroblast', 'immature granulocytes', 'lymphocyte', 'monocyte', 'neutrophil' et 'platelet'.

Dans ce notebook on va éffectué un audit général de nos images et quelques visualisation.  

# 1. Charger les données

In [ ]:
import pandas as pd
import numpy as np
from termcolor import colored
import random
from datetime import datetime

import sys
import time
import glob
import os

import cv2 ## Package pour lire nos images
from PIL import Image ## Importer ce package pour pouvoir voir le format de l'image
import skimage.exposure

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.cluster import MeanShift, estimate_bandwidth
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC 
from sklearn.metrics import classification_report

## Chemin pour aller récupérer les images stocker dans le dossier 'PBC_dataset_normal_DIB'
path ="C:/Users/HP/Desktop/Projets Data Sciences/20211121-Projet_Deep Learning_Blood_Cell/2."+\
                                           "Prepared Data/PBC_dataset_normal_DIB/"


In [ ]:
## Créer une liste où stocker les noms de nos 8 différentes classes
Classes = []
for element in os.listdir(path):
    nom_classe = os.listdir(path+element)
    Classes.append(nom_classe)

In [ ]:
# importer toutes les images des 8 différentes classes et les stocker dans un array images
Images = []
for element in os.listdir(path) :
    for img in glob.glob(path+element+'/'+'*.jpg'): 
        imge = cv2.imread(img)
        Images.append(imge)

# 2. Audit

## 2.1. Regarder la répartitions de nos données sur les 8 classes 

In [ ]:
# Afficher la taille de nos classes et le pourcentage de chacune d'entre elle dans un DataFrame
type_cellule=['basophil','eosinophil', 'erythroblast','immature granulocytes','lymphocyte','monocyte','neutrophil',
              'platelet']
Total_Number = []
Percentage = []
for i, Classe in zip(range(len(Classes)),type_cellule):
    Total_Number.append(len(Classes[i]))
    Percentage.append(round(len(Classes[i])/17092 *100,2))

dictionnaire = {'Type de Cellule': [i for i in type_cellule],
                'Total_images' : [j for j in Total_Number],
                '%' : [perc for perc in Percentage]
                 }

DF = pd.DataFrame(data = dictionnaire, columns=dictionnaire.keys())

In [ ]:
import seaborn as sns
sns.set_theme(style="whitegrid")

g = sns.catplot(y='Type de Cellule',x='Total_images',hue='Type de Cellule' ,data=DF,kind='bar',dodge=False,height=8)
g.fig.set_figwidth(12)
g.fig.set_figheight(6)
g.set_yticklabels(["BA", "EO", "ER","IG","LY","MO","NEU","PLT"])
#plt.legend(labels = [name for name in type_cellule],loc='center right',bbox_to_anchor=(1.25, 0.5), ncol=1)
plt.xlabel("Nombre d'images",size=14)
plt.ylabel("Type de Cellule",size=14);

### i. Taille des images

In [ ]:
# Taille de nos images
for img in Images :
    h, w, c = img.shape # on peut utiliser également size qui nous renvois (h*w*c)
print("height  :", h)
print("width   :", w)
print("channel :", c)

# Si jamais dans nos données on a des images avec des tailles différentes des autres, récupérer le nom des images et le 
# stocker dans une liste !
img_other_size = []    
for element, Classe in zip(os.listdir(path),Classes) :
    for img, imge in zip(glob.glob(path+element+'/'+'*.jpg'), Classe) : 
        if cv2.imread(img).shape != (h,w,c) :
            img_other_size.append(imge)

print("\n")

if not img_other_size :
    print(colored("Toutes nos images ont la même taille"),attrs=['bold'])
else :
    print(colored(len(img_other_size),attrs=['bold']),
          "images possèdent une taille différente de la taille des autres images !")

### ii. Format des images

In [ ]:
## format de nos images
for element in os.listdir(path) :
    for img in glob.glob(path+element+'/'+'*.jpg'): 
        format_image = Image.open(img).format

print("le format de nos images est :", format_image)

# Si il y'a des images avec un format différent des autres images, on va les récupérer et ensuite les stocker dans la liste
# img_other_format        
img_other_format = []

for element, Classe in zip(os.listdir(path),Classes) :
    for img, imge in zip(glob.glob(path+element+'/'+'*.jpg'), Classe) : 
        if Image.open(img).format != 'JPEG' :
            img_other_format.append(imge)

#print("\n")

if not(img_other_format) :
    print(colored("toutes nos images ont le même format !",attrs=['bold']))
else :
    img_other_format

### iii. Resizer les images qui n'ont pas la bonne dimension

In [ ]:
# dimension de nos images
height = 363
width = 360
dim = (width, height)

# on redimensionne les images une à une et on écrase celles qui n'ont pas les bonnes dimensions
for element in os.listdir(path) :
    for i in range(len(img_other_size)) :
        for j in glob.glob(path+element+'/'+img_other_size[i]): #'*.jpg'
            img = cv2.imread(j,cv2.IMREAD_UNCHANGED)
            resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
            cv2.imwrite(j,resized)
            

## 2.2. Visalisation des différents classes de cellules

In [ ]:
# Fonction pour récupérer k images de manière aléatoire
def RandomImage(k,noms_classes):
    '''
    la fonction RandomImage va nous permettre de récupérer un nombre d'images (k) de chaque classes de nos cellules
    de manière aléatoire, ensuite les stocker dans une liste img_random 
    
    paramètres :
    noms_classes : les noms de nos 8 calsses de cellules sanguines
    k : le nombre d'images qu'en veut récupérer de chaques classes de notre data base
    '''
    img_random = []
    for i in range(len(noms_classes)) :
        for j in range(k):
            img = random.choice(noms_classes[i])
            img_random.append(img) 
    return img_random

In [ ]:
#------- Afficher pour chaque type de cellules une image, qu'on récupére du manière aléatoire ---------#
# selectionner une seule cellule de chaque type de classe et les visualiser avec plt.imshow
img_random = RandomImage(1,Classes)

noms_classe = ['basophil','eosinophil','erythroblast','immature granulocytes','lymphocyte','monocyte','neutrophil',
               'platelet']

sns.set_theme(style="white")
fig= plt.figure(figsize=(18,20))   
for i in range(len(img_random)):        
    # subplots
    plt.subplot(4, 4, i+1)   
    # Afficher l'image en coleur
    plt.imshow(cv2.cvtColor(cv2.imread(path+os.listdir(path)[i]+'/'+img_random[i],cv2.IMREAD_COLOR),cv2.COLOR_BGR2RGB));
    
    # set the title to subplots
    plt.gca().set_title(noms_classe[i],size=20) ;
    

# 3. Preprocessing

## 3.1. Distributions des couleurs et la balance des blancs 

In [ ]:
# Fonction pour tracer l'histogramme des couleurs
def histogramOpenCV(img_co, img_gr, white_hi):
    _, axis = plt.subplots(nrows=2, ncols=2, figsize=(18, 10))
    axis[0][0].imshow(img_co)
    axis[0][1].set_title('Histogram')
    axis[0][0].set_title('Image')
    rgbcolors = ['red', 'green', 'blue']
    for i,col in enumerate(rgbcolors):
        histr = cv2.calcHist([img_co],[i],None,[256],[0,256])
        axis[0][1].plot(histr,color = col)
    
    axis[1][0].imshow(img_gr,'gray')
    axis[1][0].set_title("Image en noir et blanc")
    axis[1][1].plot(white_hi)
    axis[1][1].set_title("white")
    
#Lecture des images en mode noir et blanc ou couleurs
#transformation de l'image couleur en noir et blanc

# Créer des listes vide pour stocker les différentes lectures de nos images noir et blanc, coleur 
Img_gray = [] 
Img_color = [] 
White_hist = []

for i in range(len(img_random)) :
    
    img = path+os.listdir(path)[i]+'/'+img_random[i]
    
    img_gray = cv2.imread(img, cv2.IMREAD_GRAYSCALE)  # str(img)
    Img_gray.append(img_gray)
    
    img_color = cv2.imread(img, cv2.IMREAD_COLOR)  # # str(img)
    Img_color.append(img_color)
    
    img_gray2 = cv2.cvtColor(img_color, cv2.COLOR_BGR2GRAY)
    
    #histogramme en noir et blanc
    white_hist = cv2.calcHist([img_gray2],[0],None,[256],[0,256])
    White_hist.append(white_hist)

## Affichage des images
for i in range (len(img_random)):
    histogramOpenCV(Img_color[i], Img_gray[i], White_hist[i])
    
# rajouter un plot moyen  pour l'histogramme
# rajouter le plot.save

## 3.2. Transformation de l'image de RGB => HSV

In [ ]:
Hsv = [] 

#Déclaration des couleurs des courbes
color = ('r','g','b')
#Déclaration des noms des courbes.
labels = ('h','s','v')

for j in range(len(img_random)) :   
    img = path+os.listdir(path)[j]+'/'+img_random[j]
    hsv = cv2.cvtColor(cv2.imread(img), cv2.COLOR_BGR2HSV)
    Hsv.append(hsv)
    
for j in range(len(img_random)) :
    _, axis = plt.subplots(ncols=3, figsize=(18, 5))
    for i,col in enumerate(color):
        #Hist prend la valeur de l'histogramme de hsv sur la canal i.
        hist_hsv = cv2.calcHist([Hsv[j]],[i],None,[256],[0,256])

        # Plot de hist.
        #plt.subplot(223),
        axis[2].plot(hist_hsv,color = col,label=labels[i])
        axis[2].set_title('HSV')
        axis[2].legend()
        #axis[2].xlim([0,256])

    axis[0].imshow(Img_color[j], 'gray')
    axis[0].set_title("image en coleur")
    axis[1].imshow(Hsv[j], 'gray')
    axis[1].set_title('hsv');

# rajouter un plot moyen  pour l'histogramme
# rajouter le plot.save

## 3.2. Transformation de l'image de RGB => LAB


In [ ]:
Lab = []

#Déclaration des couleurs des courbes
color = ('r','g','b')
#Déclaration des noms des courbes.
labels = ('l','a','b')
for j in range(len(img_random)) :   
    img = path+os.listdir(path)[j]+'/'+img_random[j]
    lab = cv2.cvtColor(cv2.imread(img), cv2.COLOR_BGR2LAB)
    Lab.append(lab)
    
for j in range(len(img_random)) :
    _, axis = plt.subplots(ncols=3, figsize=(18, 5))
    for i,col in enumerate(color):
        #Hist prend la valeur de l'histogramme de hsv sur la canal i.
        hist_lab = cv2.calcHist([Lab[j]],[i],None,[256],[0,256])

        axis[2].plot(hist_lab,color = col,label=labels[i])
        axis[2].set_title('LAB')
        axis[2].legend()

    axis[0].imshow(Img_color[j], 'gray')
    axis[0].set_title("image en coleur")
    axis[1].imshow(Lab[j], 'gray')
    axis[1].set_title('lab');

# rajouter un plot moyen  pour l'histogramme
# rajouter le plot.save

## 3.3. Etude des contours 

###  Méthode Canny

In [ ]:
from skimage import feature

# Lire les images afficher la dessus 
imge_array = []
M = []
for i in range(len(img_random)) :
    imge = cv2.imread(path+os.listdir(path)[i]+'/'+img_random[i],cv2.IMREAD_COLOR)#,cv2.IMREAD_COLOR)
    m = np.zeros((imge.shape[0],imge.shape[1]))
    m[:,:] = imge[:,:,0]
    imge_array.append(imge)
    M.append(m)

#---------- Appliquer le filtre Canny  ----------#

edges = []
for i in range(len(imge_array)):
    edges.append(feature.canny(M[i], sigma=3)) # On peut jouer sur les valeurs de sigma  

fig = plt.figure(figsize=(18,20))   
for i in range(len(imge_array)) :        
    # subplots
    plt.subplot(4, 4, i+1)   
    # Afficher l'image
    plt.imshow(edges[i], cmap=plt.cm.gray, interpolation='nearest');  
    # set the title to subplots
    plt.gca().set_title(noms_classe[i],size=20) 
    ;

In [ ]:
# Rajouter une boucle pour faire varier le sigma (neutrophil)
# faire un filtre canny sur les sorties LAB et HSV

## 3.4. Histogramme des pixels

In [ ]:
"""fig = plt.figure(figsize=(18,20))   
for i in range(len(imge_array)) :        
    # subplots
    plt.subplot(4, 4, i+1)   
    # Afficher l'image
    plt.hist(imge_array[i].flatten(), bins = 20, density = True , alpha = .5 , edgecolor = 'black', color = 'red');  
    # set the title to subplots
    plt.gca().set_title(noms_classe[i],size=20) ;
    """


# 4. Nettoyage du Background

## Filtrage du Background en blanc 

In [ ]:
def RemoveBackground(img):
    """
    Cette fonction permis de nettoyer le background d'une image originale, elle procède comme suite au départ on éffectue un 
    GaussianBlur et en esnuite un threshold avec les résultat de threshold on crée un mask avec morphologyEx, qu'on appliquera 
    à notre image. 
    
    paramètre :: 
    img : notre image original 
    
    """
    #start = time.time()
    
    l = int(max(12,13))
    u = int(max(13,13))
     
    edges = cv2.GaussianBlur(img, (5, 5), 3) # (21, 51), 3
    edges = cv2.cvtColor(edges, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(edges, l, u)
    
    _, thresh = cv2.threshold(edges, 0, 255, cv2.THRESH_BINARY  + cv2.THRESH_OTSU)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5)) #(5, 5)
    mask = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=4)
    
    data = mask.tolist()
    sys.setrecursionlimit(10**8) #(10**8)
    for i in  range(len(data)):
        for j in  range(len(data[i])):
            if data[i][j] !=  255:
                data[i][j] =  -1
            else:
                break
        for j in  range(len(data[i])-1, -1, -1):
            if data[i][j] !=  255:
                data[i][j] =  -1
            else:
                break
    image = np.array(data)
    image[image !=  -1] =  255
    image[image ==  -1] =  0

    mask = np.array(image, np.uint8)

    img_cleaned = cv2.bitwise_and(original, original, mask=mask)
    img_cleaned[mask ==  0] =  255
    
    
    #end = time.time()
    #elapsed_time = end - start
    
    return img_cleaned #, elapsed_time

In [ ]:
Img_cleaned = []

for i in range(len(img_random)):
    img = cv2.imread(path+os.listdir(path)[i]+'/'+img_random[i])
    original = img

    img_cleaned = RemoveBackground(img)
    Img_cleaned.append(img_cleaned)
    
noms_classe = ['basophil','eosinophil','erythroblast','immature granulocytes','lymphocyte','monocyte','neutrophil',
               'platelet']

fig= plt.figure(figsize=(18,20))   
for i in range(len(img_random)):        
    # subplots
    plt.subplot(4, 4, i+1)   
    # Afficher l'image
    plt.imshow(Img_cleaned[i])
    # set the title to subplots
    plt.gca().set_title(noms_classe[i],size=20) ; 


#### La méthode de nettoyage de background en utilisant " cv2.getStructuringElement " et " cv2.morphologyEx ", ne marche pas bien sur nos images !!

##  Filtrage du background par couleur

In [ ]:
# Filtrage du background par couleur
figure = plt.figure(figsize=(18,18)) 

filtered_image = []
for i in range(len(img_random)):
    img = cv2.imread(path+os.listdir(path)[i]+'/'+img_random[i])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # on peut le faire en HSV et en LAB
    lower_color = np.array([30, 0, 0])
    upper_color = np.array([165,255,255])
    mask_color = cv2.inRange(img, lower_color, upper_color)
    res_red = cv2.bitwise_and(img,img, mask= mask_color)
    plt.subplot(4,4,i+1)
    plt.imshow(res_red);
    plt.gca().set_title(noms_classe[i],size=20)
    filtered_image.append(res_red)


#### Cette méthode donne des résultats beaucoup mieux que la précédente, on arrive à bien resortir la cellule qui nous intresse.

In [ ]:
# faire un passage en noir et blanc 

# 5. Seuillage d'OTSU

## 5.1 Seuillage d'OTSU sur les images HSV

In [ ]:
TH_h = []
TH_s = []
TH_v = []

for i in range(len(img_random)) :
    h,s,v= cv2.split(Hsv[i])
    ret_h, th_h = cv2.threshold(h,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    ret_s, th_s = cv2.threshold(s,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    ret_v, th_v = cv2.threshold(v,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    TH_h.append(th_h)
    TH_s.append(th_s)
    TH_v.append(th_v)

for j in range(len(img_random)) :
    _, axis = plt.subplots(ncols=3, figsize=(18, 5))
    axis[0].imshow(TH_h[j], 'gray')
    axis[0].set_title("Teinte Seuillée")
    axis[1].imshow(TH_s[j], 'gray')
    axis[1].set_title('Saturation Seuillée')
    axis[2].imshow(TH_v[j], 'gray')
    axis[2].set_title('Luminosité Seuillée');

# On constate que la saturation nous permet de segmenter les cellules

## 5.2. Seuillage d'OTSU sur les images LAB

In [ ]:
# créer des arrays vides, ensuite les remplire avec les images suiller 
TH_l = []
TH_a = []
TH_b = []

for i in range(len(img_random)) :
    l,a,b= cv2.split(Lab[i])
    ret_l, th_l = cv2.threshold(l,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    ret_a, th_a = cv2.threshold(a,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    ret_b, th_b = cv2.threshold(b,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    TH_l.append(th_l)
    TH_a.append(th_a)
    TH_b.append(th_b)

for j in range(len(img_random)) :
    _, axis = plt.subplots(ncols=3, figsize=(18, 5))
    axis[0].imshow(TH_l[j], 'gray')
    axis[0].set_title("Teinte Seuillée")
    axis[1].imshow(TH_a[j], 'gray')
    axis[1].set_title('Saturation Seuillée')
    axis[2].imshow(TH_b[j], 'gray')
    axis[2].set_title('Luminosité Seuillée');
    
# On constate que L et B segmentent bien les cellules

# 6. Segmenatation en utilisant K-means et Mean Shift

## 6.1. K-Means clustering avec OpenCV

###  i. Images filtrer par couleur

In [ ]:
#############----------------- K-MEANS sur les images à qui on a nettoyé le background en amont-----------------############# 

Z = [] ## Créer une liste où stocker nos images redimentionner
Clustered_img = [] 

for i in range(len(img_random)) :
    img =filtered_image[i] # Img_cleaned
    
    # redimentionner l'image en un array à deux dimension (hauteur*lrgeur,coleur)
    z = img.reshape((-1,3))   
    
    # convertir de le type de l'array en 'float32' afin que les Kmeans de OpenCV le prenne en compte
    z = np.float32(z) 
    Z.append(z)
    # définir les critéres, nombre de clusters (K) et ensuite appliquer les K-Means
    criteria = (cv2.TermCriteria_EPS + cv2.TermCriteria_MAX_ITER, 10, 1.0)
    K = 2 # on met le nombre de clusters = 2 pour juste sortir les cellule
    
    # mettre les sorties de cv2.kmeans dans rets, labels, centroids
    rets , labels , centroids = cv2.kmeans(z, K, None, criteria, 10, cv2.KMEANS_PP_CENTERS)
 
    # revenir à la dimensions originale pour pouvoir afficher l'image
    centroids = np.uint8(centroids)
    res = centroids[labels.flatten()]
    res2 = res.reshape((img.shape))
    Clustered_img.append(res2)


noms_classe = ['basophil','eosinophil','erythroblast','immature granulocytes','lymphocyte','monocyte','neutrophil',
               'platelet']

fig= plt.figure(figsize=(18,20))   
for i in range(8):        
    # subplots
    plt.subplot(4, 4, i+1)   
    # Afficher l'image  
    plt.imshow(Clustered_img[i])
    # set the title to subplots
    plt.gca().set_title(noms_classe[i],size=20) ; 


### ii. Images HSV & LAB

In [ ]:
# faire un lissage gaussien à nos images
Img_blurred = []
TH_l_blurred = []
TH_s_blurred = []
for j in range(len(img_random)) :
    img= cv2.GaussianBlur(Img_gray[j], (3, 3), 0)
    Img_blurred.append(img)
    
    th_l_blurred = cv2.GaussianBlur(TH_l[j], (3,3),0)
    TH_l_blurred.append(th_l_blurred)
    
    th_s_blurred = cv2.GaussianBlur(TH_s[j], (3,3),0)
    TH_s_blurred.append(th_s_blurred)
    
for j in range(len(img_random)) :
    _, axis = plt.subplots(ncols=3, figsize=(18, 5))
    axis[0].imshow(Img_blurred[j], 'gray')
    axis[0].set_title("Original Seuillée & Floutée")
    axis[1].imshow(TH_l_blurred[j], 'gray')
    axis[1].set_title('L Seuillée & Floutée')
    axis[2].imshow(TH_s_blurred[j], 'gray')
    axis[2].set_title('S Seuillée & Floutée');


In [ ]:
#############----------------- K-MEANS sur les images HSV et LAB-----------------############# 
# redimensionner les images
Img_blurred_rshpd = []
TH_s_blurred_rshpd = []
TH_l_blurred_rshpd = []

for j in range(len(img_random)) :
    Img_blurred_rshpd.append(Img_blurred[j].reshape(-1))
    TH_s_blurred_rshpd.append(TH_s_blurred[j].reshape(-1))
    TH_l_blurred_rshpd.append(TH_s_blurred[j].reshape(-1))

#Conversion des images de nparray en float
for j in range(len(img_random)) :
    Img_blurred_rshpd[j] = np.float32(Img_blurred_rshpd[j])
    TH_s_blurred_rshpd[j] = np.float32(TH_s_blurred_rshpd[j])
    TH_l_blurred_rshpd[j] = np.float32(TH_l_blurred_rshpd[j])

######-------------- Définition des paramètres --------------######
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 1.0)
K = 2
attempts=10

Clustered_image = []
Clustered_image_lab = []
Clustered_image_hsv = []

## Application des K-Means
for i in range(len(img_random)) :
    ret,label,center=cv2.kmeans(Img_blurred_rshpd[i],K,None,criteria,attempts,cv2.KMEANS_PP_CENTERS)
    center = np.uint8(center)
    res = center[label.flatten()]
    result_image = res.reshape((Img_blurred[j].shape))
    Clustered_image.append(result_image)
    
    ret_l,label_l,center_l=cv2.kmeans(TH_l_blurred_rshpd[i],K,None,criteria,attempts,cv2.KMEANS_PP_CENTERS)
    center_l = np.uint8(center_l)
    res_l = center[label_l.flatten()]
    result_image_lab = res_l.reshape((TH_l_blurred[i].shape))
    Clustered_image_lab.append(result_image_lab)
    
    ret_s,label_s,center_s=cv2.kmeans(TH_s_blurred_rshpd[i],K,None,criteria,attempts,cv2.KMEANS_PP_CENTERS)
    center_s = np.uint8(center_s)
    res_s = center[label_s.flatten()]
    result_image_hsv = res_s.reshape((TH_s_blurred[i].shape))
    Clustered_image_hsv.append(result_image_hsv)

for j in range(len(img_random)) :
    _, axis = plt.subplots(ncols=3, figsize=(18, 5))
    axis[0].imshow(Clustered_image[j], 'gray')
    axis[0].set_title("Original")
    axis[1].imshow(Clustered_image_lab[j], 'gray')
    axis[1].set_title('LAB')
    axis[2].imshow(Clustered_image_hsv[j], 'gray')
    axis[2].set_title('HSV');

# Graphique des centroïdes
#plt.scatter(center[:, 0],center[:, 1], marker = "o", color = "red",s=30, linewidths = 1, zorder = 10)
#plt.show()


In [ ]:
#X Y R G B
# faire un traitement d'images en X Y R G B pour la partie K-means et visualiser les centroids

## 6.2. Segmentation par Mean Shift

In [ ]:
figure = plt.figure(figsize=(18,18)) 
index_tmp = 1
#index_filtered_img= 0
for i in range(len(img_random)) :     # pour chaque index dans la liste
    img = cv2.resize(filtered_image[i],(360,360))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.subplot(4,4,index_tmp)
    plt.gca().set_title(noms_classe[i]+'- Base',size=20)
    plt.imshow(img, 'gray');
    index_tmp+=1
    
    # filtre de réduction du bruit
    img = cv2.medianBlur(img, 3)

    # aplatir l'image
    flat_image = img.reshape(img.shape[0]*img.shape[1], img.shape[2])

    # meanshift
    bandwidth = estimate_bandwidth(flat_image, quantile=0.5, n_samples=100)
    ms = MeanShift(bandwidth, max_iter=2000, bin_seeding=True)
    ms.fit(flat_image)
    labels=ms.labels_
    centroids = ms.cluster_centers_

    # Obtenir la couleur moyenne de chaque centroid
    total = np.zeros((centroids.shape[0], 3), dtype=float)
    count = np.zeros(total.shape, dtype=float)

    # Pour chaque label dans le tableau des labels
    for j, label in enumerate(labels):
        #On ajoute les valeurs RGB de chaque pixel de l'image applatit
        total[label] = total[label] + flat_image[j]
        count[label] += 1

    # On calcule la moyenne du RGB
    avg = total/count
    # On arrondi à l'entier
    avg = np.uint8(avg)

    # transposition de l'image étiquetée dans la couleur moyenne correspondante
    result = avg[labels]
    result = result.reshape((img.shape))

    #Affichage du resultat
    plt.subplot(4,4,index_tmp)
    plt.gca().set_title(noms_classe[i] + ' - MS - k' + str(centroids.shape[0]))
    plt.imshow(result, 'gray');
    index_tmp+=1